In [1]:
using Symbolics

function HM(sz::Int...; symbol::Symbol)
    (@variables $symbol[map(x -> 1:x, sz)...])[1]
end

function HM(sz::AbstractRange{Int}...; symbol::Symbol)
    (@variables $symbol[sz...])[1]
end

eltypes(::Type{Symbolics.Arr{T,D}}) where {T,D} = (T, D)
eltypes(x::Symbolics.Arr) = eltypes(typeof(x))

Base.similar(A::AbstractArray, T::Type, dims::Tuple{AbstractRange{I}, Vararg{AbstractRange{I}}}) where {I<:Int} = similar(A, T, tuple(map(x -> length(x), dims)...))
    
Base.similar(f::Union{Function,DataType}, dims::Tuple{AbstractRange{I}, Vararg{AbstractRange{I}}}) where {I<:Int} = similar(f, tuple(map(x -> length(x), dims)...))

function printHM(Hx::Symbolics.Arr{T,2}) where {T}
    println("2 method called! ", Hx)
    if any(firstindex.([Hx],[1:2...]') .!= 1)
        r,c = length.(axes(Hx))
        [Hx[i+r*j] for i in 1:r, j in 0:(c-1)]
    else
        [x for x in Hx]
    end
end

function printHM(Hx::Symbolics.Arr{T,D}, dim1=0, dim2=1, dpths=nothing) where {T,D}
    println("general method called! ", Hx)
    if any(firstindex.([Hx],[1:D...]') .!= 1)
        [Hx[i] for i in CartesianIndex(repeat([1],D)...):CartesianIndex(length.(axes(Hx))...)]
    else
        [x for x in Hx]
    end
end

printHM (generic function with 5 methods)

In [2]:
sz = 3; HV = HM(repeat([sz],sz)..., symbol=:v)
printHM(HV)

general method called! v[1:3,1:3,1:3]


3×3×3 Array{Num, 3}:
[:, :, 1] =
 v[1, 1, 1]  v[1, 2, 1]  v[1, 3, 1]
 v[2, 1, 1]  v[2, 2, 1]  v[2, 3, 1]
 v[3, 1, 1]  v[3, 2, 1]  v[3, 3, 1]

[:, :, 2] =
 v[1, 1, 2]  v[1, 2, 2]  v[1, 3, 2]
 v[2, 1, 2]  v[2, 2, 2]  v[2, 3, 2]
 v[3, 1, 2]  v[3, 2, 2]  v[3, 3, 2]

[:, :, 3] =
 v[1, 1, 3]  v[1, 2, 3]  v[1, 3, 3]
 v[2, 1, 3]  v[2, 2, 3]  v[2, 3, 3]
 v[3, 1, 3]  v[3, 2, 3]  v[3, 3, 3]

In [3]:
arguments(Symbolics.value(HV[1,1,2]))[2:end]

3-element Vector{Any}:
 1
 1
 2

In [4]:
function lexlist(sz, f)
    # it, terms = [f] .|> [enumerate length]
    sum(enumerate(f)) do (i, x)
        (x-1) * sz^(i-1)
    end
end

function lexsymbol(sz, f)
    lexlist(sz, arguments(Symbolics.value(f))[2:end])
end

lexsymbol.(sz, printHM(HV))

general method called! v[1:3,1:3,1:3]


3×3×3 Array{Int64, 3}:
[:, :, 1] =
 0  3  6
 1  4  7
 2  5  8

[:, :, 2] =
  9  12  15
 10  13  16
 11  14  17

[:, :, 3] =
 18  21  24
 19  22  25
 20  23  26

In [14]:
symbolicarray_fromsymbol(f) = arguments(Symbolics.value(f))[1]

list_fromsymbol(f) = arguments(Symbolics.value(f))[2:end]

compose_lists(f1, f2) = map(x->f1[x],f2)

# Composition is associative
compose_lists(f1, f2, fi...) = compose_lists(compose_lists(f1,f2),fi...)

function compose_symbols(f1, fi...)
    HM = symbolicarray_fromsymbol(f1)
    HM[compose_lists(([f1, fi...] .|> list_fromsymbol)...)...]
end

# f1list, f2list = [f1, f2] .|> list_fromsymbol

compose_symbols (generic function with 1 method)

In [15]:
compose_symbols.(HV[2,3,1],[HV[2,1,3], HV[1,3,2]])

2-element Vector{SymbolicUtils.Term{Real, Nothing}}:
 v[3, 2, 1]
 v[2, 1, 3]

In [16]:
compose_symbols(HV[2,3,1], HV[2,1,3], HV[1,3,2])

v[3, 1, 2]

In [10]:
function genrandfunc(sz)
    history = []
    function randFunc(ctx)
        f = rand(1:sz, sz)
        push!(history, (f,ctx))
        f
    end
    randFunc, ()->history
end
randFunc, getHistory = genrandfunc(sz)

(randFunc, var"#19#21"{Vector{Any}}(Any[]))

In [11]:
compose_symbols(HV[randFunc("cmpSym")...], HV[randFunc("cmpSym")...])

[2, 2, 2]


v[2, 2, 2]

In [12]:
getHistory()

2-element Vector{Any}:
 ([2, 2, 2], "cmpSym")
 ([3, 1, 1], "cmpSym")

In [13]:
list_fromlex(sz, terms, lex) = [((lex ÷ (sz ^ i)) % sz) + 1 for i in 0:(terms-1)]

symbol_fromlex(HM, sz, terms, lex) = HM[list_fromlex(sz, terms, lex)...]

lex_identfunc(sz) = lexlist(sz, 1:sz)

symbol_fromlex(HV, sz, sz, lex_identfunc(sz))

v[1, 2, 3]